In [ ]:
import numpy as np
import papermill as pm
import pandas as pd

In [ ]:
data_files = []
bootstraps = 1
maxiter = 1
data_dir = ''
b = []

In [ ]:
len_files = len(data_files)

for file in data_files:
    nb = pm.read_notebook(file)
    
    df = nb.dataframe
    data = df[df['name'] == 'data']['value'].values[0]
    train = data['train']
    test = data['test']
    
    for i in range(int(bootstraps/len_files)):
        #create an output notebook name to store each run's results
        output_name = (file.split('\\')[-1].split('.')[0]+'_run_'+str(i)).replace('-','_')
        output_notebook = data_dir +'/results/{}'.format(output_name+'.ipynb')
        
        #execute optimization notebook
        pm.execute_notebook(
           'optimize.ipynb',
           output_notebook,
           parameters = dict(data = train, sample = True, maxiter = maxiter, x0 = b)
        )
        
        #get coefficients
        nb = pm.read_notebook(output_notebook)
        nb_df = nb.dataframe
        x = nb_df[nb_df['name']=='x']['value'].values[0]
        
        #calculate error
        pm.execute_notebook(
           'error.ipynb',
           data_dir + '/error/{}'.format(output_name+'_error.ipynb'),
           parameters = dict(test_data = test, b = x,)
           )
        
        #get the new x0
        weighted_b = ((i+1) * np.array(b) + np.array(x)) /(i+2)
        b = list(weighted_b)
        
